# Evaluating the fine tuned model

### Needed packages and imports

In [ ]:
!pip install -r requirements.txt

### Model inference parameters

The parameters to the fine tuned model.

In [ ]:
import requests
import os
import yaml
import json
import re
import time
import pandas as pd
import torch

from typing import Iterator
from pathlib import Path
from openai import OpenAI

from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.prompts import PromptTemplate

from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.document_loaders import BaseLoader
from langchain_core.documents import Document as LCDocument

from langchain_community.llms import VLLMOpenAI
from langchain_milvus import Milvus
from langchain_text_splitters import RecursiveCharacterTextSplitter

from docling.document_converter import DocumentConverter


In [ ]:
import os

from dotenv import load_dotenv

load_dotenv()

In [ ]:
MAX_TOKENS=2048
TEMPERATURE=0.00

In [ ]:
!pip list | grep -E "docling|langchain|dotenv|torch"


### Milvus connection info

Defaults to local db

In [ ]:
MILVUS_URI = os.getenv("MILVUS_URI", "./milvus_local.db")
MILVUS_USERNAME = os.getenv("MILVUS_USERNAME", "")
MILVUS_PASSWORD = os.getenv("MILVUS_PASSWORD", "")
MILVUS_COLLECTION = os.getenv("MILVUS_COLLECTION", "my_org_documents")

## Sanity check model

In [ ]:
def create_llm(testing_config):
    return VLLMOpenAI(
        openai_api_key=re.sub(r"\s+", "", testing_config["api_key"]),
        openai_api_base=testing_config["endpoint_url"], #https://model...com/v1
        model_name=testing_config["model_name"],
        temperature=0.00,
        max_tokens=2048,
        streaming=False
    )

def qna_request(llm, template_str, question):
    num_retries = 1
    for attempt in range(num_retries):
        try:
            prompt = PromptTemplate.from_template(template_str)
            chain = prompt | llm | StrOutputParser()
            answer = chain.invoke({"question": question})        
            print(answer)
            return answer.strip()
        except Exception as e:
            print(f"Request failed: {e}")
            if attempt + 1 < num_retries:
                print(f"Retrying in 5 seconds...")
                time.sleep(5)
            else:
                return ""           
            

In [ ]:
with open("llm_config.yaml", "r") as f:
    llm_config = yaml.safe_load(f)
    
llm = create_llm(llm_config["testing_config"][0])
template_str = llm_config["testing_config"][0]["qna_template"]

question = "Which hardware accelerators are supported by RHEL AI?"
qna_request(llm, template_str, question)

## Creating an Milvus DB with documents

## Initial index creation and document ingestion

#### Load pdfs

In [ ]:
class DoclingPDFLoader(BaseLoader):

    def __init__(self, file_path: str | list[str]) -> None:
        self._file_paths = file_path if isinstance(file_path, list) else [file_path]
        self._converter = DocumentConverter()

    def lazy_load(self) -> Iterator[LCDocument]:
        for source in self._file_paths:
            dl_doc = self._converter.convert(source).document
            text = dl_doc.export_to_markdown()
            yield LCDocument(page_content=text)

In [ ]:
pdf_folder_path = "../data_preparation/document_collection"
file_paths = [str(path) for path in Path(pdf_folder_path).rglob('*.pdf')]
file_paths

In [ ]:
loader = DoclingPDFLoader(file_path=file_paths)

#### Split documents into chunks with some overlap

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1024,
    chunk_overlap=100,
)

docs = loader.load()
all_splits = text_splitter.split_documents(docs)
all_splits[0]

#### Create the index and ingest the documents

In [ ]:
device = (
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)

model_kwargs = {"trust_remote_code": True, "device": device}
embeddings = HuggingFaceEmbeddings(
    model_name="nomic-ai/nomic-embed-text-v1.5",
    model_kwargs=model_kwargs,
    show_progress=True
)

db = Milvus(
    embedding_function=embeddings,
    connection_args={
        "uri": MILVUS_URI,
        "user": MILVUS_USERNAME, 
        "password": MILVUS_PASSWORD
    },
    collection_name=MILVUS_COLLECTION,
    auto_id=True,
    drop_old=True
)


In [ ]:
loaded = db.add_documents(all_splits)
print(f"{len(loaded)} documents loaded.")

#### Test vector DB search

In [ ]:
query = "Which hardware accelerators are supported by RHEL AI?"
docs_with_score = db.similarity_search_with_score(query)

In [ ]:
for doc, score in docs_with_score:
    print("-" * 80)
    print("Score: ", score)
    print(doc.page_content)
    print("-" * 80)

#### Test out RAG request

In [ ]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 4})

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

def rag_request(llm, template_str, question):
    num_retries = 1
    for attempt in range(num_retries):
        try:
            prompt = PromptTemplate.from_template(template_str)
            rag_chain = (
                    {"context": retriever | format_docs, "question": RunnablePassthrough()}
                    | prompt
                    | llm
                    | StrOutputParser()
            )
            response = rag_chain.invoke(question)
            return response.strip()
        except Exception as e:
            print(f"Request failed: {e}")
            if attempt < num_retries:
                print(f"Retrying in 5 seconds...")
                time.sleep(5)
            else:
                return ""

In [ ]:
llm = create_llm(llm_config["testing_config"][0])
template_str = llm_config["testing_config"][0]["rag_template"]

question = "Which hardware accelerators are supported by RHEL AI?"
result = rag_request(llm, template_str, question)
result

In [ ]:
directory = "../data_preparation/document_collection"


qna_list = []

for file_path in Path(directory).rglob('qna.yaml'):
    print(file_path)
    if not file_path.name == 'qna.yaml':
        continue
    with open(file_path) as file:
        qna = yaml.load(file, Loader=yaml.FullLoader)
        for seed_example in qna["seed_examples"]:
            for questions_and_answers in seed_example["questions_and_answers"]:
                qna_list.append(
                    {
                        "question": questions_and_answers["question"].strip(),
                        "ground_truth": questions_and_answers["answer"].strip()                     
                    }
                )
                
# print(qna_list)

qna_df = pd.DataFrame(qna_list)
# df.to_csv('qna.csv', index=False)
qna_df.to_json('qna.jsonl', orient='records', lines=True)


In [ ]:
import yaml

def replace_special_char(original_str):
    return re.sub(r"[^\w]", "_", original_str)

with open("llm_config.yaml", "r") as f:
    llm_config = yaml.safe_load(f)
    
qna_df = pd.read_json("qna.jsonl", orient="records", lines=True)

for testing_config in llm_config["testing_config"]:
    answers = qna_df.copy()
    answers["answer"] = ""
    answers["rag_answer"] = ""
    llm = create_llm(testing_config)
    for index, row in answers.iterrows():
        question = row["question"]
        print(index, question)
        if testing_config.get("qna_template"):
            answer = qna_request(llm, testing_config.get("qna_template"), question)
            print("QnA Answer: " + answer[:40])
            answers.at[index, "answer"] = answer
        if testing_config.get("rag_template"):
            answer = rag_request(llm, testing_config.get("rag_template"), question)
            print("RAG Answer: " + answer[:40])
            answers.at[index, "rag_answer"] = answer
    base_filename = replace_special_char(testing_config["name" or "model_name"])
    answers.to_json(f"{base_filename}_answers.jsonl", orient="records", lines=True)
    # answers.to_csv(f"{base_filename}_answers.csv")

## Grade responses using Judge Model

In [ ]:
import yaml

with open("llm_config.yaml", "r") as f:
    llm_config = yaml.safe_load(f)

In [ ]:
from langchain.prompts import PromptTemplate

SCORING_PROMPT = PromptTemplate(
    template="""You are an evaluation system tasked with assessing the answer quality of a AI generated response in relation to the posed question and reference answer. Assess if the response is correct, accurate, and factual based on the reference answer. Evaluate the answer_quality as:
    - Score 1: The response is completely incorrect, inaccurate, and/or not factual.
    - Score 2: The response is mostly incorrect, inaccurate, and/or not factual.
    - Score 3: The response is somewhat correct, accurate, and/or factual.
    - Score 4: The response is mostly correct, accurate, and factual.
    - Score 5: The response is completely correct, accurate, and factual.
    Here is the question: \n ------- \n {question} \n -------
    Here is model answer: \n ------- \n {answer} \n -------
    Here is the reference answer(may be very short and lack details or indirect, long and extractive):  \n ------- \n {reference_answer} \n ------- \n
    Assess the quality of model answer with respect to the Reference Answer, but do not penalize the model answer for adding details or give a direct answer to user question. Provide the quality level as a JSON object with two keys: 'reasoning' and 'answer_quality'.
    """,
    input_variables=["question", "answer", "reference_answer"],
)


In [ ]:
judge_client = OpenAI(api_key=llm_config["judge"]["api_key"])
judge_model_name = llm_config["judge"]["model_name"]


def score_request(question, answer, reference_answer):
    messages = [
        {
            "role": "user",
            "content": SCORING_PROMPT.format(
                question=question,
                answer=answer,
                reference_answer=reference_answer
            )
        }
    ]

    completion = judge_client.chat.completions.create(
        model=judge_model_name,
        messages=messages,
        n=1,
        temperature=0.0,
        max_tokens=1024,
    )

    response_content = completion.choices[0].message.content
    result = json.loads(response_content)
    score = result["answer_quality"]
    reasoning = result["reasoning"]
    return score, reasoning


In [ ]:
for testing_config in llm_config["testing_config"]:
    base_filename = replace_special_char(testing_config["name" or "model_name"])
    answers_filename = f"{base_filename}_answers.jsonl"
    scores = pd.read_json(answers_filename, orient="records", lines=True)
    position = scores.columns.get_loc("answer")
    scores.insert(position + 1, "answer_score", "")
    scores.insert(position + 2, "answer_score_reasoning", "")
    position = scores.columns.get_loc("rag_answer")
    scores.insert(position + 1, "rag_answer_score", "")
    scores.insert(position + 2, "rag_answer_score_reasoning", "")

    for index, row in scores.iterrows():
        question = row["question"]
        answer = row["answer"]
        reference_answer = row["ground_truth"]
        print(index, question)
        if answer:
            score, reasoning = score_request(question, answer, reference_answer)
            scores.at[index, "answer_score"] = score
            scores.at[index, "answer_score_reasoning"] = reasoning
            print(answer[:40], score, reasoning[:40])
        rag_answer = row["rag_answer"]
        if rag_answer:
            score, reasoning = score_request(question, rag_answer, reference_answer)
            scores.at[index, "rag_answer_score"] = score
            scores.at[index, "rag_answer_score_reasoning"] = reasoning
            print(rag_answer[:40], score, reasoning[:40])

    judge_filename = replace_special_char(judge_model_name)
    scores_filename = f"{base_filename}_{judge_filename}_scores.jsonl"
    scores.to_json(scores_filename, orient="records", lines=True)
    scores.to_csv(f"{base_filename}_{judge_filename}_scores.csv", index=False)


## Create resulting score report CSV

In [ ]:
with open("llm_config.yaml", "r") as f:
    llm_config = yaml.safe_load(f)

judge_client = OpenAI(api_key=llm_config["judge"]["api_key"])
judge_model_name = llm_config["judge"]["model_name"]
judge_filename = replace_special_char(judge_model_name)

summary_output_df = pd.DataFrame()

for testing_config in llm_config["testing_config"]:
    base_filename = replace_special_char(testing_config["name" or "model_name"])
    scores_filename = f"{base_filename}_{judge_filename}_scores.jsonl"
    scores = pd.read_json(scores_filename, orient="records", lines=True)
    if testing_config.get("qna_template"):
        summary_output_df[f"{base_filename}_answer_score"] = scores["answer_score"]
    if testing_config.get("rag_template"):
        summary_output_df[f"{base_filename}_rag_answer_score"] = scores["rag_answer_score"]


summary_output_df.to_json(f"summary_{judge_filename}_scores.jsonl", orient="records", lines=True)
summary_output_df.to_csv(f"summary_{judge_filename}_scores.csv", index=False)